In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Local module
from nxr_classes import Organisations

In [4]:
nbr = pd.read_excel("nbr - alle organisasjoner.xlsx")

#### Nasjonalt barnehageregister (NBR)
Udirs beskrivelse av registeret:   
"Data fra Nasjonalt barnehageregister viser enheter fra Virksomhets- og Foretaksregisteret (VoF) som tilhører barnehage sektoren. Barnehagene (bedriftene/virksomhetene) og deres eiere (foretak) skal være avstemt med VoF. NBR viser også informasjon som kommer fra Barnehage-Statistikk-Innsamling-Løsningen (Basil)."

Vi har hentet ut detaljert informasjon om samtlige 14 927 organisasjoner som er registrert i NBR.

I registeret er det registrert:
- Organisasjonsnummer
- Navn
- Barnehageeier
- Beliggenhet
- Kontaktinformasjon (epost, telefon og postadresse)
- Målform
- Organisasjonsform
- Kontaktinformasjon til barnehagestyrer
- Antall barn
- Antall ansatte
- Alderstrinn (fra/til)
- Næringskoder
- Barnehagekategorier

Dette dokumentet går gjennom den mest sentrale informasjon i nbr. 

##### Aktive og nedlagte barnehager

Først og fremst er det verdt å merke seg at NBR består av både aktive og nedlagte organsisasjoner. Tabellen under viser forskjellige status i registeret og hvor mange barnehager som tilhører hver av dem. I NBR er det 9 522 aktive barnehager, eller sagt på registerspråket: Barnehager uten utgåtttype. 

In [13]:
nbr['Utgaattype_Navn'].value_counts()

Ingen utgåttype                       9522
Slettet                               4715
Slettet for sammenslåing               362
Utg pga eier mangler aktiv bedrift     174
Utg pga næringsendring                 111
Slettet for dublett                     42
Name: Utgaattype_Navn, dtype: int64

Det er også mulig å identifisere nedlagte organisasjoner med kolonnnen `ErAktiv`.

In [14]:
pd.crosstab(nbr.Utgaattype_Navn, nbr.ErAktiv)

ErAktiv,False,True
Utgaattype_Navn,,
Ingen utgåttype,20,9502
Slettet,4715,0
Slettet for dublett,42,0
Slettet for sammenslåing,362,0
Utg pga eier mangler aktiv bedrift,174,0
Utg pga næringsendring,111,0


##### Identifisere barnehagene
Det er ikke alle organisasjoner i NBR som er en barnehage. 8 806 av organisasjonene er registrert som barnehage, og samtlige av dem står oppført med organisasjonsformen `Bedrift`. Organisasjoner registrert i NBR som ikke er barnehager er ofte barnehageeiere.  

In [15]:
pd.crosstab(nbr.Organisasjonsform_Navn, nbr.ErBarnehage)

ErBarnehage,False,True
Organisasjonsform_Navn,,
Aksjeselskap,1918,0
Allment aksjeselskap,1,0
Annen juridisk person,27,0
Annet foretak iflg. særskilt lov,35,0
Ansvarlig selskap,78,0
Ansvarlig selskap med delt deltakeransvar,136,0
Bedrift,348,8806
Borettslag,5,0
Den norske kirke,90,0


In [16]:
pd.crosstab(nbr.ErBarnehage, nbr.ErBarnehageEier)

ErBarnehageEier,False,True
ErBarnehage,,
False,1758,4362
True,8806,0


Fra nå av fokuserer vi bare på organisasjonene registrert som aktive barnehager. Det er 5 955 organisasjoner som er registrert som aktive barnehager.  

In [17]:
pd.crosstab(nbr.ErBarnehage, nbr.ErAktiv)

ErAktiv,False,True
ErBarnehage,,
False,2573,3547
True,2851,5955


In [19]:
bhg = nbr.query('ErAktiv == True & ErBarnehage == True')

##### Barnehagekategori

Barnehagene i NBR er registrert med forskjellige kategorier. Brukte kategorier, er: 

- Offentlig barnehage
- Privat barnehage
- Inaktiv i Basil

Offentlig og privat barnehage er gjensidig utelukkende kategorier, men bagge har noe overlapp med inaktivitet i Basil. Hva som kjennetegner en aktiv barnehage i NBR som er inaktiv i Basil er ukjent. 

In [21]:
pd.crosstab(bhg.ErPrivatBarnehage, bhg.ErOffentligBarnehage)

ErOffentligBarnehage,False,True
ErPrivatBarnehage,,
False,0,2818
True,3137,0


In [22]:
pd.crosstab(bhg.ErPrivatBarnehage, bhg.ErInaktivIBasil)

ErInaktivIBasil,False,True
ErPrivatBarnehage,,
False,2650,168
True,2886,251


In [24]:
pd.crosstab(bhg.ErPrivatBarnehage, bhg.ErOffentligBarnehage)

ErOffentligBarnehage,False,True
ErPrivatBarnehage,,
False,0,2818
True,3137,0


Ut fra barnehagekategoriene er det mulig å identifisere kommunale barnehager, familiebarnehager, åpne barnehager, mm.  

265 av barnehagene har bare én kategori, og det er med ett unntak "Privat barnehage". 


In [27]:

bhg_bhgkat = ["Orgnr"] + [col for col in bhg.columns if col.startswith("Barnehagekategorier") and col.endswith("_Navn")]

kategorier = bhg[bhg_bhgkat]

kategorier = pd.melt(kategorier, id_vars ="Orgnr", value_vars=bhg_bhgkat[1:])
kategorier = kategorier[kategorier["value"].notna()]
kategorier.sort_values(by = "Orgnr")

kategorier["value"].value_counts()



Ordinær barnehage           4971
Privat barnehage            3137
Offentlig barnehage         2818
Kommunal barnehage          2805
Familiebarnehage             391
Åpen barnehage               104
Kombinert barnehage           36
Statlig barnehage             11
Fylkeskommunal barnehage       1
Name: value, dtype: int64

In [28]:
kategorier["count"] = kategorier.groupby("Orgnr")['value'].transform(len)
kategorier_one = kategorier.query("count == 1")
kategorier_one["value"].value_counts()


Privat barnehage       264
Offentlig barnehage      1
Name: value, dtype: int64